In [1]:
from time import perf_counter_ns
import numpy as np
from scipy.sparse import csc_matrix, kron, identity
import cirq
import openfermion as of
from openfermionpyscf import run_pyscf
import quimb.tensor as qtn
from convert import to_groups_of
from error_pert import get_v2_sarray
from qpe_trotter import v2_pauli_sum
from kcommute import get_si_sets
from tensor_network_common import pauli_sum_to_mpo, mps_to_vector

In [2]:
molec = "LiH"
basis = "sto-3g"
n_elec = 4
geometry = of.chem.geometry_from_pubchem(molec)
multiplicity = 1
molecule = of.chem.MolecularData(
    geometry, basis, multiplicity
)
molecule = run_pyscf(molecule, run_scf=1, run_fci=1)
print(f"HF energy:", molecule.hf_energy)
print(f"FCI energy:", molecule.fci_energy)
hamiltonian = molecule.get_molecular_hamiltonian()
hamiltonian_qubop = of.transforms.jordan_wigner(hamiltonian)
hamiltonian_psum = of.transforms.qubit_operator_to_pauli_sum(hamiltonian_qubop)

nq = of.utils.count_qubits(hamiltonian_qubop)
nterms = len(hamiltonian_qubop.terms)
print(f"Hamiltonian has {nq} qubits and {nterms} terms.")

HF energy: -7.767362135748557
FCI energy: -7.784460280031223
Hamiltonian has 12 qubits and 631 terms.


In [3]:
qs = cirq.LineQubit.range(nq)
hamiltonian_mpo = pauli_sum_to_mpo(hamiltonian_psum, qs, 100)
dmrg = qtn.DMRG(hamiltonian_mpo, bond_dims=15)
converged = dmrg.solve()
if not converged:
    print("DMRG did not converge.")
ground_state = dmrg.state
ground_state_vec = mps_to_vector(ground_state)

/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/cotengra/hyperoptimizers/hyper.py:55: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization. It is recommended to install one of these libraries for higher quality contraction paths.
  warnings.warn(
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: divide by zero encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: overflow encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: invalid value encountered in matmul
  return func(*args, **kwargs)


sorted_inds = ['k0', 'k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7', 'k8', 'k9', 'k10', 'k11']


/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: divide by zero encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: overflow encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: invalid value encountered in matmul
  return func(*args, **kwargs)


In [4]:
groups = get_si_sets(hamiltonian_psum, nq)
print(f"There are {len(groups)} groups.")

There are 42 groups.


In [5]:
# Use the code from the paper.
start_time = perf_counter_ns()
group_qubops = to_groups_of(groups)
sparse_frag_ops = []
# Convert group operators to sparse matrices.
# We must make sure the matrices have all the same size.
# If a matrix is not big enough, tensor it with I on the right.
for op in group_qubops:
    nq_op = of.utils.count_qubits(op)
    op_sparse = of.linalg.get_sparse_operator(op)
    if nq_op != nq:
        eye_diff = identity(2 ** (nq - nq_op), dtype="complex", format='csc')
        new_op = kron(op_sparse, eye_diff, format="csc")
        sparse_frag_ops.append(new_op)
    else:
        sparse_frag_ops.append(op_sparse)
v2_sparse = get_v2_sarray(sparse_frag_ops)
eps2 = np.vdot(ground_state_vec, v2_sparse @ ground_state_vec).real
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"Got eps2={eps2} in {elapsed_time:4.5e} ns")

Got eps2=0.00976985337076401 in 1.67026e+09 ns


In [6]:
start_time = perf_counter_ns()
group_psums = [sum(group) for group in groups]
v2_psum = v2_pauli_sum(group_psums)
qubit_map = {q: i for i, q in enumerate(qs)}
eps2_psum = v2_psum.expectation_from_state_vector(ground_state_vec, qubit_map)
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"Got eps2={eps2_psum} in {elapsed_time:4.5e} ns")

Got eps2=(0.009769853370764012+4.643960537537107e-21j) in 5.50096e+11 ns
